In [1]:
import os

In [2]:
# Check present working directory
%pwd

'c:\\Users\\deept\\ShopTalk\\research'

In [3]:
# Change present working directory to thr root directory
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\deept\\ShopTalk'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    untar_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from pathlib import Path

In [7]:
class ConfigurationManager:
    CONFIG_FILE_PATH = r'C:\Users\deept\ShopTalk\config\config.yaml'
    PARAMS_FILE_PATH = r"C:\Users\deept\ShopTalk\params.yaml"
    SCHEMA_FILE_PATH = r"C:\Users\deept\ShopTalk\schema.yaml"
    def __init__(
        self,
        config_filepath = Path(CONFIG_FILE_PATH),
        params_filepath = Path(PARAMS_FILE_PATH),
        schema_filepath = Path(SCHEMA_FILE_PATH)):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir, 
            untar_path=config.untar_path 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
import tarfile
import gzip
import time
from mlProject import logger
from mlProject.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    def cleanup_files(self, file_paths):
        """
        Attempts to clean up files that could not be deleted during the extraction process.
        """
        for file_path in file_paths:
            try:
                os.remove(file_path)
                print(f"File {file_path} deleted successfully during cleanup.")
            except PermissionError as e:
                print(f"Failed to delete {file_path} during cleanup: {e}")   
    
    def extract_compressed_files_from_tar(self):
        # List to keep track of files that could not be deleted
        files_to_cleanup = []
        """
        Extracts all folders/files inside 'data.tar' which could be compressed by any format.
        """
        # Path to the 'data.tar' file
        tar_file_path = Path(self.config.unzip_dir) / 'data.tar'
        
        # Create a directory to extract the contents of 'data.tar'
        tar_extract_dir = tar_file_path.parent / 'data_tar_extracted'
        os.makedirs(tar_extract_dir, exist_ok=True)
        
        # Extract the 'data.tar' file
        with tarfile.open(tar_file_path, 'r') as tar:
            tar.extractall(path=tar_extract_dir)
        
        # Walk through the directory where 'data.tar' was extracted
        for root, dirs, files in os.walk(tar_extract_dir):
            for file in files:
                file_path = Path(root) / file
                
                # Handle gzip compression
                if file_path.suffix == '.gz':
                    with gzip.open(file_path, 'rb') as f_in:
                        output_file_path = file_path.with_suffix('')
                        with open(output_file_path, 'wb') as f_out:
                            f_out.write(f_in.read())
                            try:
                                os.remove(file_path)
                            except PermissionError as e:
                                print(f"Could not delete {file_path}: {e}")
                                files_to_cleanup.append(file_path)
                # Handle zip compression
                elif file_path.suffix == '.zip':
                    with zipfile.ZipFile(file_path, 'r') as zip_ref:
                        zip_ref.extractall(path=file_path.parent)
                    os.remove(file_path)  # Remove the .zip file after extraction
        self.cleanup_files(files_to_cleanup)       



In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_compressed_files_from_tar()
except Exception as e:
    raise e

[2024-04-23 22:30:38,724: INFO: common: yaml file: C:\Users\deept\ShopTalk\config\config.yaml loaded successfully]
[2024-04-23 22:30:38,731: INFO: common: yaml file: C:\Users\deept\ShopTalk\params.yaml loaded successfully]
[2024-04-23 22:30:38,748: INFO: common: yaml file: C:\Users\deept\ShopTalk\schema.yaml loaded successfully]
[2024-04-23 22:30:38,750: INFO: common: created directory at: artifacts]
[2024-04-23 22:30:38,751: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-23 22:30:38,754: INFO: 2070140361: File already exists of size: ~ 85430 KB]
Could not delete artifacts\data_ingestion\data_tar_extracted\listings\metadata\listings_0.json.gz: [WinError 32] The process cannot access the file because it is being used by another process: 'artifacts\\data_ingestion\\data_tar_extracted\\listings\\metadata\\listings_0.json.gz'
Could not delete artifacts\data_ingestion\data_tar_extracted\listings\metadata\listings_1.json.gz: [WinError 32] The process cannot access the